In [7]:
import zipfile 
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MKkipZP0knNNm_6EO9jRshxAo3Du1Hom' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1MKkipZP0knNNm_6EO9jRshxAo3Du1Hom" -O data.zip && rm -rf /tmp/cookies.txt

with zipfile.ZipFile('./data.zip', 'r') as zip_ref:
        print('The extraction of the images can take some time...')
        zip_ref.extractall('./')
        print('Extraction done')

--2019-12-08 13:49:57--  https://docs.google.com/uc?export=download&confirm=3Krg&id=1MKkipZP0knNNm_6EO9jRshxAo3Du1Hom
Resolving docs.google.com (docs.google.com)... 172.217.20.14, 2a00:1450:400d:805::200e
Connecting to docs.google.com (docs.google.com)|172.217.20.14|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mjl8egm51559spsap78q8q3cm9g5tf2s/1575806400000/17506844314399870089/*/1MKkipZP0knNNm_6EO9jRshxAo3Du1Hom?e=download [following]
--2019-12-08 13:49:57--  https://doc-08-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mjl8egm51559spsap78q8q3cm9g5tf2s/1575806400000/17506844314399870089/*/1MKkipZP0knNNm_6EO9jRshxAo3Du1Hom?e=download
Resolving doc-08-24-docs.googleusercontent.com (doc-08-24-docs.googleusercontent.com)... 216.58.214.193, 2a00:1450:400d:802::2001
Connecting to doc-08-24-docs.googleusercontent.com (doc-08-24

In [3]:
import os
import math, random
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train = "./cleaned_data/train/"
validation = "./cleaned_data/validation/"

aug_train = "./augmented_data/train"
aug_validation = "./augmented_data/validation"

try:
    categories = os.listdir(train)
    for c in categories:
        os.makedirs(os.path.join(aug_train,c))
        os.makedirs(os.path.join(aug_validation,c))
except OSError:
    print ("Necessary directories already exist")
else:
    print ("Successfully created the directories")

Successfully created the directories


In [4]:
def si_augment(path, to_where, datagen, over_add=0, over_perc=0, max_corr=False):
    #get all categories from path
    categories = os.listdir(path)
    print("Augmentation of",categories, "classes has begun. It may take some time...")
    #number of images in each category
    image_nums = [len([name for name in os.listdir(data)]) 
                  for data in [path+cat for cat in categories]]
    #what would be the ideal amount of image for each categories
    if not max_corr:
        idealcount = (math.ceil(sum(image_nums)/len(categories))+over_add)*(1+over_perc)
    else:
        idealcount = math.ceil(sum(image_nums)/len(categories))+math.ceil((max(image_nums)*0.15))
    #how many image is missing for the ideal number
    to_gener = [abs(num-idealcount) for num in image_nums]
    where = 0
    for cat in categories:
        #if we have less image then needed we need to generate
        if idealcount > image_nums[where]:
            #lets calculate how many copy needed for every image in the given category
            #estim_num = math.ceil(to_gener[where]/image_nums[where])
            print("Squared image augmentation for image class",
                  cat, "with", to_gener[where],"image(s)")
            #from where we load the images
            src = os.path.join(path,cat)
            images = os.listdir(src)
            save_dir = os.path.join(to_where,cat)
            #IMPORTANT!!! since the image generator tries to avoid duplication of images
            #it will not allow us to create two images that are the same (even by random)
            #we have to generate until there is the right amount of pictures
            while (len([name for name in os.listdir(save_dir)])!=to_gener[where]):
                img = load_img(os.path.join(src, random.choice(images)))
                x = img_to_array(img)
                x = x.reshape((1,) + x.shape)
                x.shape
                gen_img = datagen.flow(x,
                                   batch_size=1,
                                   save_to_dir=save_dir,
                                   save_prefix='aug', 
                                   save_format='png')
                for x in gen_img:
                    break
            #after generating the necessary images, let's copy the original ones
            for image in images:
                full_file_name = os.path.join(src, image)
                shutil.copy(full_file_name, save_dir)
        #if there is more image then needed 
        #we copy the ideal amount of image to the destination folder
        else:
            print("Copying", idealcount, "image(s) for class", cat)
            for k in range(idealcount):
                which_img = os.path.join(path,cat,os.listdir(os.path.join(path,cat))[k])
                aug_img = os.path.join(to_where,cat,os.listdir(os.path.join(path,cat))[k])
                shutil.copyfile(which_img, aug_img)
        where+=1
    print("Image augmentation done!")


datagen = ImageDataGenerator(rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

si_augment(train, aug_train, datagen, max_corr=True)
#si_augment(validation, aug_validation, datagen)

Augmentation of ['concrete_cement', 'healthy_metal', 'incomplete', 'irregular_metal', 'other'] classes has begun. It may take some time...
Squared image augmentation for image class concrete_cement with 1254 image(s)
Copying 1834 image(s) for class healthy_metal
Squared image augmentation for image class incomplete with 1517 image(s)
Copying 1834 image(s) for class irregular_metal
Squared image augmentation for image class other with 1773 image(s)
Image augmentation done!


In [5]:
#image_nums = [len([name for name in os.listdir(data)]) 
#                  for data in [train+cat for cat in categories]]
#idealcount = math.ceil(sum(image_nums)/len(categories))+math.ceil((max(image_nums)*0.15))
#print(idealcount)

print(len(os.listdir(os.path.join(aug_train,categories[0]))))
print(len(os.listdir(os.path.join(aug_train,categories[2]))))
print(len(os.listdir(os.path.join(aug_train,categories[4]))))

1834
1834
1834


In [18]:
from PIL import Image

paths = [train, validation]
sizes = []
for p in paths:
    for cat in categories:
        location = os.path.join(p,cat)
        for file in os.listdir(location):
            im = Image.open(os.path.join(location,file))
            sizes.append(im.size)

In [21]:
import numpy as np
sizes = np.array(sizes)
np.std(sizes)
np.mean(sizes)

233.03112233528933

In [33]:
paths = [os.path.join('./cleaned_data',p) for p in os.listdir('./cleaned_data')]
numbers = [math.ceil(len(os.listdir(x))/2) for x in paths]
categories = ['concrete_cement','healthy_metal','incomplete','irregular_metal','other']
where = 0
print(numbers)
for p in paths:
    counter = 0
    for file in os.listdir(paths[where]):
        filename = os.path.join(p,file)
        halfname = os.path.join("./halfed",categories[where],file)
        shutil.copyfile(filename, halfname)
        counter+=1
        if counter is numbers[where]:
            break
    counter = 0
    for file in os.listdir(paths[where]):
        filename = os.path.join(p,file)
        os.remove(filename)
        counter+=1
        if counter is numbers[where]:
            break
    where += 1
    print(p, "done")

[580, 3252, 318, 2521, 62]
./cleaned_data/concrete_cement done
./cleaned_data/healthy_metal done
./cleaned_data/incomplete done
./cleaned_data/irregular_metal done
./cleaned_data/other done
